In [ ]:
scope.dis()
target.dis()

In [1]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
scope.adc.samples=24400

OSError: Unable to communicate with found ChipWhisperer. Check that 
another process isn't connected to it and that you have permission to communicate with it.

In [ ]:
# UNMASKED
# function
def setup():
    print(sum(p1.puf))
    # runs once in the start -> prints returned
    load("unmasked")
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand
    if op in ["random", "tvla"]:
        p_temp = [random.randrange(2) for _ in range(127)]
    elif op == "set":
        p_temp = p1.puf
    elif op == "zero":
        p_temp = p0.puf
    else:
        return None
    mem = xor(p_temp, s.encode(k))
    # posture('k', get_bytes(mem), 16)
    p_temp = xor(p_temp, mem) # comment if mem is included
    return get_bytes(p_temp)

def ending(fdb, op, t):
    # runs after each CW command -> True <=> trace isnt taken
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (get_bits(fdb)[:64] != k):
        print("WRONG!")
        return True
    pass

# operation settings
root = "F:/programs/traces"
operations = ["random", "tvla", "set", "zero"] # operations list
subfolder = "unmasked" # subfolder name
TRACES_NUM = 1000 # number of traces
scope.adc.samples = 20000 # number of samples per trace
simulate = False # doesnt do anything for now. should be False
s = Scheme() # scheme execution helper

rand = None
p0 = PUF([0 for _ in range(127)]) #PUF([random.randrange(2) for _ in range(127)])
p1 = PUF([random.randrange(2) for _ in range(127)]) #PUF([random.randrange(2) for _ in range(127)])
k = [random.randrange(2) for _ in range(64)]
set_weight = sum(p1.puf)
key_weight = sum(k)

print(setup())
time.sleep(0.1)
start_time = date_str()
##
file_names = [f"file{i}" for i in range(2)]
for file_name in file_names:
    status = traces(file_name, 0)
split_file(file_names, [0, 1], "random")
split_file(file_names, [0, 2], "set")
split_file(file_names, [0, 3], "zero")
remove_files(file_names)
##
generate_file("start_time", "operations", "set_weight", "key_weight")
print("done")